In [ ]:
# Montando Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Definindo diretório padrão deste notebook
import sys
base_path = "/content/drive/MyDrive/VAE_TCC"
sys.path.append(base_path)

In [ ]:
# Instalando bibliotecas auxiliares
!pip install tensorflow
!pip install soundfile
!pip install auraloss
!pip install librosa

In [ ]:
# Importando bibliotecas auxiliares
import time
import numpy as np
import soundfile as sf
import tensorflow as tf
from tensorflow.keras.backend import clear_session

In [ ]:
# Importando arquivos auxiliares
import utils
import configs
from model import VAE_GAN
from audio_info import AudioInfo

In [ ]:
# Re-carregando arquivos para garantia de alterações
import importlib
importlib.reload(utils)
importlib.reload(configs)

<module 'configs' from '/content/drive/MyDrive/VAE_TCC/configs.py'>

In [ ]:
start_time = time.time()
current_id = utils.generate_random_id()
print(f"[ID do Experimento] - {current_id}")

[ID do Experimento] - GX9266


In [ ]:
# Carregando os dados de áudio e instanciando otimizador
audio_info = AudioInfo(audio_path=configs.AUDIO_PATH, duration=configs.AUDIO_DURATION, sr=configs.AUDIO_RATE)

In [ ]:
# Extraindo e preparando dados de Treinamento
data = audio_info.get_audio_data(num_audio_samples=configs.NUM_AUDIO_SAMPLES)
multiband_dataset = []
for i in range(data.shape[0]):
    subbands = audio_info.multiband_decomposition(data[i], num_bands=configs.NUM_BANDS)
    multiband_dataset.append(subbands)

data = np.array(multiband_dataset)
print(f"[Dados de Treinamento] - Formato: {data.shape}")

[Extraindo formas de onda]
 - ethereal: OK
 - hype: OK
 - how_sweet: OK
 - fancy: OK
 - supernatural: OK
 - goth: OK

[Dados de Treinamento] - Formato: (6, 41250, 16)


In [ ]:
# Inicializando o modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=configs.LEARNING_RATE, beta_1=configs.BETA_1,
                                     beta_2=configs.BETA_2, epsilon=configs.EPSILON)
model = VAE_GAN(input_shape=data.shape, latent_dim=configs.LATENT_DIM, hidden_dims=configs.VAE_HIDDEN_DIMS, id=current_id,
            duration=configs.AUDIO_DURATION, rate=configs.AUDIO_RATE, kernel_sizes=configs.VAE_KERNELS, strides=configs.VAE_STRIDES,
             loud_stride=configs.LOUD_STRIDE, kl_beta=configs.KL_BETA, batch_size=configs.BATCH_SIZE, num_bands=configs.NUM_BANDS,
            residual_depth=configs.RESIDUAL_DEPTH, use_noise=configs.USE_NOISE)


[Incializando VAE-GAN]
	[CONSTRUINDO ENCODER]
		 - Forma de saída MU:  (None, 323, 16)
		 - Forma de saída LOG_VAR:  (None, 323, 16)

	[CONSTRUINDO DECODER]
	[INFO]Formato de entrada:  (6, 41250, 16)
	[INFO]Comprimento do Encoder:  323
	[INFO]Fator de Decodificação:  128
	[INFO]Unidades do Decoder:  165376
	[INFO] Cropping necessário (41344 != 41250)
		 - Forma de saída DECODER:  (None, 41250, 16)
[VAE-GAN Inicializado]


In [ ]:
# Visualizando modelo
model.summary()

Model: "vae_gan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Encoder (Functional)                 │ [(None, 323, 16), (None,    │         912,160 │
│                                      │ 323, 16)]                   │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Decoder (Functional)                 │ (None, 41250, 16)           │       8,319,840 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,232,000 (35.22 MB)

 Trainable params: 9,231,072 (35.21 MB)

 Non-trainable params: 928 (3.62 KB)

In [ ]:
model.encoder.summary()

Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 41250, 16)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 41250, 16)      │             64 │ input_layer[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu (LeakyReLU)   │ (None, 41250, 16)      │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 10313, 64)      │          8,256 │ leaky_re_lu[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 10313, 64)      │            256 │ conv1d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU) │ (None, 10313, 64)      │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 2579, 128)      │         65,664 │ leaky_re_lu_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 2579, 128)      │            512 │ conv1d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_2 (LeakyReLU) │ (None, 2579, 128)      │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 645, 256)       │        262,400 │ leaky_re_lu_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 645, 256)       │          1,024 │ conv1d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_3 (LeakyReLU) │ (None, 645, 256)       │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 323, 512)       │        524,800 │ leaky_re_lu_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_4 (LeakyReLU) │ (None, 323, 512)       │              0 │ conv1d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 323, 16)        │         40,976 │ leaky_re_lu_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 323, 16)        │          8,208 │ leaky_re_lu_4[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 912,160 (3.48 MB)

 Trainable params: 911,232 (3.48 MB)

 Non-trainable params: 928 (3.62 KB)

In [ ]:
model.decoder.summary()

Model: "Decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 323, 16)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ upsampling_block          │ (None, 1292, 512)      │              0 │ input_layer_1[0][0]    │
│ (UpsamplingBlock)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ residual_block            │ (None, 1292, 512)      │      6,292,992 │ upsampling_block[0][0] │
│ (ResidualBlock)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ upsampling_block_1        │ (None, 5168, 256)      │              0 │ residual_block[0][0]   │
│ (UpsamplingBlock)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ residual_block_1          │ (None, 5168, 256)      │      1,573,632 │ upsampling_block_1[0]… │
│ (ResidualBlock)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ upsampling_block_2        │ (None, 20672, 128)     │              0 │ residual_block_1[0][0] │
│ (UpsamplingBlock)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ residual_block_2          │ (None, 20672, 128)     │        393,600 │ upsampling_block_2[0]… │
│ (ResidualBlock)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ upsampling_block_3        │ (None, 41344, 64)      │              0 │ residual_block_2[0][0] │
│ (UpsamplingBlock)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ residual_block_3          │ (None, 41344, 64)      │         49,344 │ upsampling_block_3[0]… │
│ (ResidualBlock)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_18 (Conv1D)        │ (None, 41344, 16)      │          7,184 │ residual_block_3[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_19 (Conv1D)        │ (None, 41344, 16)      │          3,088 │ residual_block_3[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 41344, 16)      │              0 │ conv1d_18[0][0],       │
│                           │                        │                │ conv1d_19[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cropping1d (Cropping1D)   │ (None, 41250, 16)      │              0 │ multiply[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 8,319,840 (31.74 MB)

 Trainable params: 8,319,840 (31.74 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Treinando a representação do modelo
clear_session()
signal_losses, kl_losses = model.train(data, configs.EPOCHS, optimizer)

[Iniciando Treinamento de Representação]
[INFO] Dataset não dividido em Batches para o treinamento.

# [ Epoca 1 | Loss: 12.0887852 |  Recon. Loss: 7.1295156 | KL Loss: 49.5926971]
# [ Epoca 16 | Loss: 1.2737327 |  Recon. Loss: 0.818414 | KL Loss: 4.5531869]
# [ Epoca 31 | Loss: 1.3838789 |  Recon. Loss: 0.7761124 | KL Loss: 6.0776658]
# [ Epoca 46 | Loss: 0.8866318 |  Recon. Loss: 0.7925036 | KL Loss: 0.941282]
# [ Epoca 61 | Loss: 1.2133524 |  Recon. Loss: 0.7510482 | KL Loss: 4.6230416]
# [ Epoca 76 | Loss: 0.8076826 |  Recon. Loss: 0.777316 | KL Loss: 0.3036664]
# [ Epoca 91 | Loss: 0.9099581 |  Recon. Loss: 0.7456765 | KL Loss: 1.6428159]
# [ Epoca 106 | Loss: 0.9752117 |  Recon. Loss: 0.7747583 | KL Loss: 2.0045338]
# [ Epoca 121 | Loss: 0.7159551 |  Recon. Loss: 0.703907 | KL Loss: 0.1204815]
# [ Epoca 136 | Loss: 0.7595376 |  Recon. Loss: 0.7152879 | KL Loss: 0.442497]
# [ Epoca 151 | Loss: 0.8921537 |  Recon. Loss: 0.7590299 | KL Loss: 1.3312384]
# [ Epoca 166 | Loss: 0.743069

In [ ]:
# Treinando o modelo com adversarial fine-tuning
clear_session()
gen_optimizer = tf.keras.optimizers.Adam(learning_rate=configs.LEARNING_RATE, beta_1=configs.BETA_1,
                                     beta_2=configs.BETA_2, epsilon=configs.EPSILON)
discr_optimizer = tf.keras.optimizers.Adam(learning_rate=configs.LEARNING_RATE, beta_1=configs.BETA_1,
                                     beta_2=configs.BETA_2, epsilon=configs.EPSILON)
generator_losses, discriminator_losses = model.train_gan(data, configs.DISCR_EPOCHS, gen_optimizer, discr_optimizer,
                                                         configs.DISCR_HIDDEN_DIMS, configs.DISCR_KERNELS, configs.DISCR_STRIDES)

[Iniciando Ajuste Fino Adversarial]
[INFO] Dataset não dividido em Batches para o treinamento.

# [Epoca 1 | Generator Loss: 0.9159315824508667 | Discriminator Loss: 1.9981598854064941]
# [Epoca 16 | Generator Loss: 4.914133071899414 | Discriminator Loss: 0.0]
# [Epoca 31 | Generator Loss: 4.911765098571777 | Discriminator Loss: 0.0]
# [Epoca 46 | Generator Loss: 4.911764621734619 | Discriminator Loss: 0.0]
# [Epoca 61 | Generator Loss: 4.911764621734619 | Discriminator Loss: 0.0]
# [Epoca 76 | Generator Loss: 4.911764621734619 | Discriminator Loss: 0.0]
# [Epoca 91 | Generator Loss: 4.911764621734619 | Discriminator Loss: 0.0]
# [Epoca 106 | Generator Loss: 4.911764621734619 | Discriminator Loss: 0.0]
# [Epoca 121 | Generator Loss: 4.911764621734619 | Discriminator Loss: 0.0]
# [Epoca 136 | Generator Loss: 4.911764621734619 | Discriminator Loss: 0.0]
# [Epoca 151 | Generator Loss: 4.911764621734619 | Discriminator Loss: 0.0]
# [Epoca 166 | Generator Loss: 4.911764621734619 | Discrimin

In [ ]:
end_time = time.time()
execution_time = end_time - start_time
execution_time = str(round(execution_time, 2))

In [ ]:
# Obtendo medias MU para visualização do espaço latente
mu = model.encode(data)[0]
mu = mu.numpy()

In [ ]:
# Avaliando espaço latente
reduced_latent = model.compact_latent_representation(mu)

[Dimensões informativas selecionadas: 13 de 16]

[Dimensões informativas: (13, 16)]


In [ ]:
utils.show_results(execution_time)
#utils.save_metadata(current_id, execution_time)
utils.save_graphs(current_id, signal_losses, kl_losses, generator_losses, discriminator_losses, mu, configs.COMPACT_LATENT_SPACE)

[TREINAMENTOS CONCLUIDOS]
	 - Tempo de execução: 2471.01 segundos
[Graficos salvos em /content/drive/MyDrive/VAE_TCC/graphs/GX9266.png]



In [ ]:
# Gerando N formas de ondas a partir do espaço latente
generated = model.sample(configs.NUM_SAMPLES_GENERATE, data, configs.COMPACT_LATENT_SPACE)

[Amostragem de espaço latente COMPACTO]
	[INFO]Formato do espaço latente compacto: (6, 323, 13)
	[INFO]Dimensões informativas: 13
	[INFO]Amostras de espaço latente compacto: (4, 323, 13)
	[INFO]Amostras finais geradas: (4, 41250, 16)


In [ ]:
# Para cada espectrograma gerado, salva o resultado em formato de áudio e em um arquivo txt
for i in range(configs.NUM_SAMPLES_GENERATE):
    curr_gen = generated[i]
    curr_gen = audio_info.multiband_synthesis(curr_gen, configs.NUM_BANDS)
    file_result = base_path + '/results/generated_audio_' + current_id + '_' + str(i+1) + '.ogg'
    sf.write(file_result, curr_gen, configs.AUDIO_RATE)
    print(f"[ Salvando resultado gerado {i + 1} ] - {current_id}")

[ Salvando resultado gerado 1 ] - GX9266
[ Salvando resultado gerado 2 ] - GX9266
[ Salvando resultado gerado 3 ] - GX9266
[ Salvando resultado gerado 4 ] - GX9266
